1. Number of updated projects of both sheets since and before (by donor agency and by SWG)(The number of updated projects refers to both the online Excel sheet and Google form)
2. Number of updated projects on the online Excel sheet only (whose online form is not updated)( by donor agency and by SWG)
3. Number of new projects and list registered ( by donor agency and by SWG)
4. Number of not updated projects ( by donor agency and by SWG)
5. Can we get an Excel sheet under each SWG updated and not updated project info of both online and Excel sheet, updated fields colored?
6. Can we get all SWG info in one file in the above similar request?

On the AMP system or new system or online form: create a links for the user like existing since, existing before, new project, select donor org (select yours in the case of multi), etc. when all infor filled filter out specific project and take the data encoder to relevant page, meaning page with filtered out projects.

In [1]:
import pandas as pd
import re
from openpyxl.styles import PatternFill, Border, Side, Alignment, Font
# improt tqdm for progress bar
from tqdm import tqdm

# Define your functions get_ids_and_titles and compare_stat here
# get_ids_and_titles
def get_ids_and_titles(original, latest):
    # Retrieve values from specific columns
    original_titles = original.iloc[7:, 2].values
    latest_titles = latest.iloc[7:, 2].values
    original_ids = original.iloc[7:, 7].values
    latest_ids = latest.iloc[7:, 7].values

    return original_titles, latest_titles, original_ids, latest_ids   

# compare_stat
def compare_stat(original, latest):
    # 3. Create dictionaries for categorizing projects
    new_projs = {}
    original_not_updated_projs = {}
    original_updated_projs = {}
    deleted_projs = {}

    _, latest_titles, original_ids, latest_ids = get_ids_and_titles(original, latest)

    # get all ids which are in original but not in latest
    for i, original_id in enumerate(original_ids):
        if original_id not in latest_ids:
            deleted_projs[original_id] = original.iloc[i + 7]

    # 4. Iterate over latest IDs for comparison
    for i, latest_id in enumerate(latest_ids): 
        # declare empty index and row variables   

        if latest_id in original_ids:
            # Find the index of the row with the matching ID in original DataFrame
            original_index = original[original.iloc[:, 7] == latest_id].index[0]
            original_row = original.iloc[original_index]

            # Extract and compare the entire rows
            latest_index = latest[latest.iloc[:, 7] == latest_id].index[0]
            latest_row = latest.iloc[latest_index]

            # Categorize based on comparison
            if original_row.equals(latest_row):
                original_not_updated_projs[latest_id] = original_row
            else:
                original_updated_projs[latest_id] = latest_row

        # enter here if id null or space or special characters in the ids excluding hyphen '-', id not in 87... or new-... or empty
        elif pd.isna(latest_id) or latest_id.isspace() or bool(re.search(r'[^a-zA-Z0-9\-]', latest_id)) or latest_id is '':
            if latest_titles[i] not in new_projs: # latest_titles[i] not in new_project
                # new_project as id: corresponding row including values of all columns
                new_projs[latest_titles[i]] = latest.iloc[i + 7]
            new_projs[latest_titles[i]] = latest.iloc[i + 7]
        else:
            # print id and say id error
            print(f'###### ID ERROR. Id: {latest_id} Title: {latest_titles[i]} ######')                       

    return new_projs, deleted_projs, original_not_updated_projs, original_updated_projs

def process_files(orig_file, lat_file, output_fname):
    original_sinc = pd.read_excel(orig_file, sheet_name=sincesht)
    latest_sinc = pd.read_excel(lat_file, sheet_name=sincesht)

    original_befo = pd.read_excel(orig_file, sheet_name=beforesht)
    latest_befo = pd.read_excel(lat_file, sheet_name=beforesht)

    original_new = pd.read_excel(orig_file, sheet_name=newsht)
    latest_new = pd.read_excel(lat_file, sheet_name=newsht)

    # get comparison statistics
    new_project_sinc, deleted_project_sinc, original_not_updated_projs_sinc, original_updated_projs_sinc = compare_stat(original_sinc, latest_sinc)
    new_project_befo, deleted_project_befo, original_not_updated_projs_befo, original_updated_projs_befo = compare_stat(original_befo, latest_befo)
    new_project_new, deleted_project_new, original_not_updated_projs_new, original_updated_projs_new = compare_stat(original_new, latest_new)

    
    # write to excel
    with pd.ExcelWriter(output_fname) as writer:
        pd.DataFrame.from_dict(new_project_sinc, orient='index').to_excel(writer, sheet_name='New Projs Since 2017')
        pd.DataFrame.from_dict(deleted_project_sinc, orient='index').to_excel(writer, sheet_name='Deleted Projs Since 2017')
        pd.DataFrame.from_dict(original_updated_projs_sinc, orient='index').to_excel(writer, sheet_name='Updated Projs Since 2017')
        pd.DataFrame.from_dict(original_not_updated_projs_sinc, orient='index').to_excel(writer, sheet_name='Not Updated Projs Since 2017')

        pd.DataFrame.from_dict(new_project_befo, orient='index').to_excel(writer, sheet_name='New Projs Before 2017')
        pd.DataFrame.from_dict(deleted_project_befo, orient='index').to_excel(writer, sheet_name='Deleted Projs Before 2017')
        pd.DataFrame.from_dict(original_updated_projs_befo, orient='index').to_excel(writer, sheet_name='Updated Projs Before 2017')
        pd.DataFrame.from_dict(original_not_updated_projs_befo, orient='index').to_excel(writer, sheet_name='Not Updated Projs Before 2017')

        pd.DataFrame.from_dict(new_project_new, orient='index').to_excel(writer, sheet_name='New Projs new')
        # pd.DataFrame.from_dict(deleted_project_new, orient='index').to_excel(writer, sheet_name='Deleted Projs new')
        # pd.DataFrame.from_dict(original_updated_projs_new, orient='index').to_excel(writer, sheet_name='Updated Projs new')
        # pd.DataFrame.from_dict(original_not_updated_projs_new, orient='index').to_excel(writer, sheet_name='Not Updated Projs new')
        
    # read excel file back and change the column names for all sheets
    df = pd.read_excel(output_fname, sheet_name=None)

    sheets_to_exclude = []
    for sheet in df:
        # Check if the sheet has zero columns before slicing
        if df[sheet].shape[1] == 0:
            sheets_to_exclude.append(sheet)
            continue

        # Now safe to remove the first two and the last one columns
        df[sheet] = df[sheet].iloc[:, 2:-1]

        # Further checks and adjustments
        if df[sheet].shape[1] > len(cols):
            # Slice to only the first 19 columns if there are more
            df[sheet] = df[sheet].iloc[:, :len(cols)]
        elif df[sheet].shape[1] < len(cols):
            # Add missing columns if the DataFrame has fewer than 19 columns
            for col in cols[df[sheet].shape[1]:]:
                df[sheet][col] = pd.NA

        df[sheet].columns = cols

    # Concatenate specific sheets into one DataFrame
    sheets_to_merge = ['New Projs Since 2017', 'New Projs Before 2017', 'New Projs new']
    merged_df = pd.concat([df[s] for s in sheets_to_merge if s not in sheets_to_exclude], axis=0)
    df['New Projects'] = merged_df[cols]

    # Exclude original sheets that were merged
    sheets_to_exclude.extend(sheets_to_merge)

    # write to excel
    with pd.ExcelWriter(output_fname) as writer:
        for sheet in df:
            if sheet in sheets_to_exclude:
                print(f'###### Empty Sheet {sheet} has been excluded ######')
                continue
            df[sheet].to_excel(writer, sheet_name=sheet, index=False)
            # Access the workbook and the sheet
            
            worksheet = writer.sheets[sheet]

            # highlight the header cells
            # Define header format
            header_fill = PatternFill(start_color="D7E4BC", end_color="D7E4BC", fill_type="solid")
            thin_border = Border(left=Side(style='thin'), 
                                right=Side(style='thin'), 
                                top=Side(style='thin'), 
                                bottom=Side(style='thin'))
            header_font = Font(bold=True)
            header_alignment = Alignment(horizontal='center', vertical='top', wrap_text=True)

            # Apply header format
            for cell in worksheet[1]:
                cell.fill = header_fill
                cell.border = thin_border
                cell.font = header_font
                cell.alignment = header_alignment

            # add filter to the header
            worksheet.auto_filter.ref = "A1:S1"

            # freeze the first row
            worksheet.freeze_panes = 'A2' 

            # Set column widths
            high = ['A', 'B']  
            medium = ['F', 'G', 'H', 'L', 'M', 'K',  'I', 'J', 'Q', 'R', 'S', 'T'] 
            low = ['C', 'D', 'E', 'N', 'O', 'P']

            for col in high:
                worksheet.column_dimensions[col].width = 30

            for col in medium:
                worksheet.column_dimensions[col].width = 18 

            for col in low:
                worksheet.column_dimensions[col].width = 12    

# Define column names and sheet names
cols = ['SWG', 'Project Title', 'Status', 'On/Off/Treasury Budget', 'Humanitarian Aid', 'Project description', 
        'Project ID (in AMP)', 'Actual Approval Date', 'Actual Start Date', 'Actual Closure Date', 
        'Donor Group', 'Donor Agency', 'Implementing Agency', 'Type of Assistance', 'Mode of Payment', 
        'Aid Modality', 'Actual Commitments', 'Actual Disbursements', 'Undisbursed Balance']

sincesht = 'Existing projects approved sinc'
beforesht = 'Existing projects approved befo'
newsht = 'Record new projects'

# List of file pairs and their corresponding output files
file_pairs = [
    ('ODA_Original_and_Latest/OrigHPN.xlsx', 'ODA_Original_and_Latest/LatHPN.xlsx', 'Report/HPN.xlsx'),
    ('ODA_Original_and_Latest/OrigBSD.xlsx', 'ODA_Original_and_Latest/LatBSD.xlsx', 'Report/BSD.xlsx'),
    ('ODA_Original_and_Latest/OrigREDFS.xlsx', 'ODA_Original_and_Latest/LatREDFS.xlsx', 'Report/REDFS.xlsx'),
    # Add more file pairs as needed
]

tqdm.write('Processing files...')
for orig_file, lat_file, output_fname in tqdm(file_pairs):
    process_files(orig_file, lat_file, output_fname)
tqdm.write('Done!')    


<>:53: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:53: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\wesag\AppData\Local\Temp\ipykernel_22124\3595527233.py:53: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif pd.isna(latest_id) or latest_id.isspace() or bool(re.search(r'[^a-zA-Z0-9\-]', latest_id)) or latest_id is '':


Processing files...


  0%|          | 0/3 [00:00<?, ?it/s]

###### Empty Sheet New Projs Since 2017 has been excluded ######
###### Empty Sheet New Projs Before 2017 has been excluded ######
###### Empty Sheet Deleted Projs Before 2017 has been excluded ######
###### Empty Sheet New Projs new has been excluded ######


 67%|██████▋   | 2/3 [00:10<00:04,  4.73s/it]

###### Empty Sheet New Projs Since 2017 has been excluded ######
###### Empty Sheet Deleted Projs Since 2017 has been excluded ######
###### Empty Sheet New Projs Before 2017 has been excluded ######
###### Empty Sheet Deleted Projs Before 2017 has been excluded ######
###### Empty Sheet Updated Projs Before 2017 has been excluded ######
###### Empty Sheet New Projs new has been excluded ######
###### Empty Sheet New Projs Since 2017 has been excluded ######
###### Empty Sheet Deleted Projs Since 2017 has been excluded ######
###### Empty Sheet New Projs Before 2017 has been excluded ######
###### Empty Sheet Deleted Projs Before 2017 has been excluded ######
###### Empty Sheet Updated Projs Before 2017 has been excluded ######
###### Empty Sheet New Projs new has been excluded ######


100%|██████████| 3/3 [00:17<00:00,  5.73s/it]

Done!


In [3]:
# Merge all Excel files in a directory into one
import pandas as pd
import os
from glob import glob

# Directory containing Excel files
directory = 'Report'

# Find all Excel files in the directory
excel_files = glob(os.path.join(directory, '*.xlsx'))

# Dictionary to store data from each sheet
merged_data = {}

# Process each file
for file in excel_files:
    # Read each sheet in the file
    xls = pd.ExcelFile(file)
    for sheet_name in xls.sheet_names:
        # Read the sheet
        df = pd.read_excel(file, sheet_name=sheet_name)
        
        # If the sheet name is already in merged_data, append the new data
        if sheet_name in merged_data:
            merged_data[sheet_name] = pd.concat([merged_data[sheet_name], df])
        else:
            merged_data[sheet_name] = df

# Write merged data to a new Excel file
with pd.ExcelWriter('Merged.xlsx') as writer:
    for sheet_name, data in merged_data.items():
        data.to_excel(writer, sheet_name=sheet_name, index=False)

In [2]:
# # Tabular
# import pandas as pd
# from tabulate import tabulate

# # Load the Excel file
# df = pd.read_excel('Report/HPN.xlsx', sheet_name=None)

# # Loop through each sheet and print a tabular summary for column L
# for sheet_name, sheet_data in df.items():
#     if sheet_data.shape[1] == 0:
#         print(f'###### Empty Sheet {sheet_name} has been excluded ######\n')
#         continue

#     # Check if column L exists and get its name
#     column_L_index = 11  # Index of column L
#     if column_L_index < len(sheet_data.columns):
#         column_L_name = sheet_data.columns[column_L_index]

#         # Group by column L and count the occurrences
#         category_counts = sheet_data.groupby(sheet_data[column_L_name]).size()

#         # Convert to DataFrame for tabulate
#         category_counts_df = category_counts.reset_index()
#         category_counts_df.columns = [column_L_name, 'Count']

#         # Printing the counts in tabular format
#         print(f"Distribution in {sheet_name} by {column_L_name}:")
#         print(tabulate(category_counts_df, headers='keys', tablefmt='grid'), "\n")
